# Data_Cleaning_Customer_Data_Using_Python_Project

In [1]:

import pandas as pd
import numpy as np
import re


In [5]:

# Load Dataset
df = pd.read_excel("Super_Messy_60Kplus_EDA_Dataset.xlsx")
df.head()


,Order ID,Customer Name,Order Date,Category,Quantity,Price,Discount %,State,Payment Mode,Rating
0,13582,Rahul,07-12-23,Clothing,-3,NaN,10,TAMIL NADU,Credit Card,3
1,11346,NaN,2023-01-15,???,1,1999.99,ten,unknown,NaN,5
2,15218,kiran,32/13/2023,NaN,two,499.5,-5%,karnataka,Cash,2
3,17763,Rahul,2023/04/01,NaN,4,₹2500,10,unknown,credit card,five
4,19785,ANJALI,32/13/2023,HOME,five,-150,15,TAMIL NADU,credit card,5


In [9]:

# Basic Inspection
df.shape
df.info()
df.isnull().sum()
df.duplicated().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65000 entries, 0 to 64999
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Order ID       65000 non-null  object
 1   Customer Name  56856 non-null  object
 2   Order Date     56943 non-null  object
 3   Category       55622 non-null  object
 4   Quantity       57709 non-null  object
 5   Price          56866 non-null  object
 6   Discount %     56917 non-null  object
 7   State          56962 non-null  object
 8   Payment Mode   55864 non-null  object
 9   Rating         57818 non-null  object
dtypes: object(10)
memory usage: 5.0+ MB


np.int64(5000)

In [10]:

# Remove Completely Empty Rows
df = df.dropna(how='all')

# Remove Duplicates
df = df.drop_duplicates()


In [11]:

# Standardize Column Names
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
df.columns


Index(['order_id', 'customer_name', 'order_date', 'category', 'quantity',
       'price', 'discount_%', 'state', 'payment_mode', 'rating'],
      dtype='object')

In [12]:

# Clean Text Columns
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].astype(str).str.strip()


In [13]:

# Clean Order Date
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')


C:\Users\adada\AppData\Local\Temp\ipykernel_24468\305800275.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')


In [14]:

# Clean Quantity
df['quantity'] = df['quantity'].replace({'two': 2, 'five': 5})
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')
df.loc[df['quantity'] < 0, 'quantity'] = np.nan


In [15]:

# Clean Price
df['price'] = df['price'].astype(str)
df['price'] = df['price'].str.replace("₹", "", regex=False)
df['price'] = df['price'].str.replace(",", "", regex=False)
df['price'] = df['price'].replace("free", 0)
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df.loc[df['price'] < 0, 'price'] = np.nan


In [16]:

# Clean Discount
df['discount_%'] = df['discount_%'].astype(str)
df['discount_%'] = df['discount_%'].str.replace("%", "", regex=False)
df['discount_%'] = df['discount_%'].replace("ten", 10)
df['discount_%'] = pd.to_numeric(df['discount_%'], errors='coerce')
df.loc[df['discount_%'] < 0, 'discount_%'] = np.nan


In [17]:

# Clean Rating
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
df.loc[~df['rating'].between(1,5), 'rating'] = np.nan


In [18]:

# Clean State
df['state'] = df['state'].str.lower().str.strip()
df['state'] = df['state'].replace({'unknown': np.nan, '123': np.nan})


In [19]:

# Clean Payment Mode
df['payment_mode'] = df['payment_mode'].str.lower().str.strip()
df['payment_mode'] = df['payment_mode'].replace({
    'cod': 'cash on delivery',
    'cash': 'cash on delivery',
    '??': np.nan
})


In [20]:

# Feature Engineering
df['final_price'] = df['price'] * df['quantity']
df['discount_amount'] = df['final_price'] * (df['discount_%'] / 100)
df['net_amount'] = df['final_price'] - df['discount_amount']


In [21]:

# Final Missing Value Handling
df['discount_%'] = df['discount_%'].fillna(0)
df['rating'] = df['rating'].fillna(df['rating'].median())

df = df.dropna(subset=['order_date','price','quantity'])


In [23]:

# Final Check
#df.info()
#df.describe()
df.isnull().sum()


order_id              0
customer_name         0
order_date            0
category              0
quantity              0
price                 0
discount_%            0
state              5401
payment_mode       3087
rating                0
final_price           0
discount_amount    5447
net_amount         5447
dtype: int64

After Cleaning You Should Have:

✔ No duplicates
✔ No empty rows
✔ Correct date format
✔ Numeric columns properly converted
✔ No negative prices
✔ Standardized categories
✔ Valid ratings (1–5)
✔ Clean state names